# 🛒 Amazon Scraper

Test all Amazon scraper endpoints:
- Product scraping (URL-based)
- Reviews scraping (URL-based)
- Seller scraping (URL-based)
- Product search (keyword-based)

---

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

API_TOKEN = os.getenv("BRIGHTDATA_API_TOKEN")
if not API_TOKEN:
    raise ValueError("Set BRIGHTDATA_API_TOKEN in .env file")

print(f"API Token: {API_TOKEN[:10]}...{API_TOKEN[-4:]}")

# Check SDK version and location
import brightdata
print(f"SDK Version: {brightdata.__version__}")
print(f"SDK Location: {brightdata.__file__}")
print("Setup complete!")

API Token: 7011787d-2...3336
SDK Version: 2.1.2
SDK Location: /Users/ns/Desktop/projects/sdk-python/src/brightdata/__init__.py
Setup complete!


## Initialize Client

In [2]:
from brightdata import BrightDataClient

client = BrightDataClient(token=API_TOKEN)

print("Client initialized")

Client initialized


---
## Test 1: Product Scraping (Single URL)

Scrape a single Amazon product by URL.

In [6]:
PRODUCT_URL = "https://www.amazon.com/dp/B0CRMZHDG8"

print(f"Scraping product: {PRODUCT_URL}\n")

async with client:
    result = await client.scrape.amazon.products(url=PRODUCT_URL)

print(f"Success: {result.success}")
print(f"Status: {result.status}")

if result.success and result.data:
    data = result.data
    print("\n=== Product Data ===")
    print(f"Title: {data.get('title', 'N/A')}")
    print(f"Brand: {data.get('brand', 'N/A')}")
    print(f"Price: {data.get('final_price', data.get('final_price_high', 'N/A'))}")
    print(f"Rating: {data.get('rating', 'N/A')}")
    print(f"Reviews: {data.get('reviews_count', 'N/A')}")
    print(f"ASIN: {data.get('asin', 'N/A')}")
    print(f"\n=== All Available Fields ({len(data)} fields) ===")
    for key, value in data.items():
        val_str = str(value)[:80] + "..." if len(str(value)) > 80 else str(value)
        print(f"  {key}: {val_str}")
else:
    print(f"\nError: {result.error}")

Scraping product: https://www.amazon.com/dp/B0CRMZHDG8

Success: True
Status: ready

=== Product Data ===
Title: STANLEY Quencher H2.0 Tumbler with Handle and Straw 40 oz | Flowstate 3-Position Lid | Cup Holder Compatible for Travel | Insulated Stainless Steel Cup | BPA-Free | Fuchsia
Brand: STANLEY
Price: 45
Rating: 4.7
Reviews: 2231
ASIN: B0CRMZHDG8

=== All Available Fields (80 fields) ===
  title: STANLEY Quencher H2.0 Tumbler with Handle and Straw 40 oz | Flowstate 3-Position...
  seller_name: Amazon.com
  brand: STANLEY
  description: Constructed of recycled stainless steel for sustainable sipping, our 40 oz Quenc...
  initial_price: 45
  currency: USD
  availability: Only 1 left in stock - order soon.
  reviews_count: 2231
  categories: ['Home & Kitchen', 'Kitchen & Dining', 'Storage & Organization', 'Thermoses', 'I...
  asin: B0CRMZHDG8
  buybox_seller: Amazon.com
  number_of_sellers: 1
  root_bs_rank: 18687
  answered_questions: 0
  domain: https://www.amazon.com/
  images_cou

---
## Test 2: Product Scraping (Batch URLs)

Scrape multiple Amazon products at once.

In [7]:
PRODUCT_URLS = [
    "https://www.amazon.com/dp/B0CRMZHDG8",
    "https://www.amazon.com/dp/B081FVQCHQ",
    "https://www.amazon.com/dp/B00LZWV8JO",
]

print(f"Batch scraping {len(PRODUCT_URLS)} products...\n")

async with client:
    results = await client.scrape.amazon.products(url=PRODUCT_URLS)

print(f"Results: {len(results)} responses\n")

for i, result in enumerate(results):
    print(f"=== Product {i+1} ===")
    print(f"  URL: {PRODUCT_URLS[i]}")
    print(f"  Success: {result.success}")
    if result.success and result.data:
        data = result.data
        print(f"  Title: {data.get('title', 'N/A')[:60]}...")
        print(f"  Price: {data.get('final_price', data.get('final_price_high', 'N/A'))}")
        print(f"  Rating: {data.get('rating', 'N/A')}")
    else:
        print(f"  Error: {result.error}")
    print()

Batch scraping 3 products...

Results: 3 responses

=== Product 1 ===
  URL: https://www.amazon.com/dp/B0CRMZHDG8
  Success: True
  Title: Apple 96W USB-C Power Adapter...
  Price: 59.99
  Rating: 4.8

=== Product 2 ===
  URL: https://www.amazon.com/dp/B081FVQCHQ
  Success: True
  Title: Seveneves: A Novel...
  Price: 15.99
  Rating: 4.3

=== Product 3 ===
  URL: https://www.amazon.com/dp/B00LZWV8JO
  Success: True
  Title: STANLEY Quencher H2.0 Tumbler with Handle and Straw 40 oz | ...
  Price: 45
  Rating: 4.7



---
## Test 3: Reviews Scraping

Scrape product reviews from an Amazon product URL.

In [8]:
PRODUCT_URL = "https://www.amazon.com/dp/B081FVQCHQ"

print(f"Scraping reviews for: {PRODUCT_URL}\n")

async with client:
    result = await client.scrape.amazon.reviews(url=PRODUCT_URL)

print(f"Success: {result.success}")
print(f"Status: {result.status}")

if result.success and result.data:
    # Reviews come as a list or dict
    reviews = result.data if isinstance(result.data, list) else [result.data]
    print(f"\nFound {len(reviews)} review(s):\n")
    
    # Show first review with all fields
    if reviews:
        print("=== First Review - All Fields ===")
        for key, value in reviews[0].items():
            val_str = str(value)[:100] + "..." if len(str(value)) > 100 else str(value)
            print(f"  {key}: {val_str}")
else:
    print(f"\nError: {result.error}")

Scraping reviews for: https://www.amazon.com/dp/B081FVQCHQ

Success: True
Status: ready

Found 27 review(s):

=== First Review - All Fields ===
  url: https://www.amazon.com/dp/B081FVQCHQ
  product_name: Apple 96W USB-C Power Adapter
  product_rating: 4.8
  product_rating_object: {'one_star': 48, 'two_star': 0, 'three_star': 48, 'four_star': 168, 'five_star': 2133}
  product_rating_max: 5
  rating: 5
  author_name: MG
  asin: B081FVQCHQ
  product_rating_count: 2397
  review_header: Very functional
  review_id: R3FVXJ3FNNW2M1
  review_text: Not sure when you’ll ever need this much power but it’s great
  author_id: AHECVLK6K72UNXV4PNJRNAA3GVJQ
  author_link: https://www.amazon.com/gp/profile/amzn1.account.AHECVLK6K72UNXV4PNJRNAA3GVJQ/ref=cm_cr_dp_d_gw_tr?ie...
  badge: Verified Purchase
  brand: Apple
  review_posted_date: January 20, 2026
  review_country: United States
  helpful_count: 0
  is_amazon_vine: False
  is_verified: True
  variant_asin: None
  variant_name: None
  videos: Non

---
## Test 4: Seller Scraping

Scrape seller information from an Amazon seller page URL.

**Note:** Use seller page format `amazon.com/sp?seller=SELLER_ID`, not store pages.

In [3]:
# Seller page URL format: amazon.com/sp?seller=SELLER_ID
# Note: Store pages (amazon.com/stores/...) are different from seller pages
SELLER_URL = "https://www.amazon.com/sp?seller=A2R2RITDJNW1Q6"

print(f"Scraping seller: {SELLER_URL}\n")

async with client:
    result = await client.scrape.amazon.sellers(url=SELLER_URL)

print(f"Success: {result.success}")
print(f"Status: {result.status}")

if result.success and result.data:
    data = result.data
    print("\n=== Seller Data ===")
    print(f"Name: {data.get('name', data.get('seller_name', 'N/A'))}")
    print(f"Rating: {data.get('rating', data.get('seller_rating', 'N/A'))}")
    print("\n=== All Available Fields ===")
    for key, value in data.items():
        val_str = str(value)[:80] + "..." if len(str(value)) > 80 else str(value)
        print(f"  {key}: {val_str}")
else:
    print(f"\nError: {result.error}")

Scraping seller: https://www.amazon.com/sp?seller=A2R2RITDJNW1Q6

Success: True
Status: ready

=== Seller Data ===
Name: N/A
Rating: N/A

=== All Available Fields ===
  timestamp: 2026-02-02T11:32:43.446Z
  input: {'url': 'https://www.amazon.com/sp?seller=A2R2RITDJNW1Q6'}
  error: Seller A2R2RITDJNW1Q6 no more exist on https://www.amazon.com
  error_code: dead_page


---
## Test 5: Product Search (Keyword-based)

Search for Amazon products by keyword and filters.

In [4]:
KEYWORD = "macbook charger"

print(f"Searching products: '{KEYWORD}'\n")

async with client:
    result = await client.search.amazon.products(
        keyword=KEYWORD,
        timeout=300  # 5 minutes
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")

if result.success and result.data:
    products = result.data if isinstance(result.data, list) else [result.data]
    print(f"\nFound {len(products)} product(s):\n")
    
    for i, product in enumerate(products[:5]):
        print(f"=== Product {i+1} ===")
        print(f"  Title: {product.get('title', 'N/A')[:60]}...")
        print(f"  Price: {product.get('final_price', product.get('price', 'N/A'))}")
        print(f"  Rating: {product.get('rating', 'N/A')}")
        print(f"  URL: {product.get('url', 'N/A')}")
        print()
else:
    print(f"\nError: {result.error}")

Searching products: 'macbook charger'

Success: True
Status: ready

Found 594 product(s):

=== Product 1 ===
  Title: N/A...
  Price: 26.83
  Rating: 4.4
  URL: https://www.amazon.com/Mac-Book-Pro-Charger-Cable%EF%BC%887-2ft/dp/B0B1HJ666G/ref=sr_1_1_sspa?dib=eyJ2IjoiMSJ9.ruSU4Gshj7Qto5dVEGD7y5KR1OH1-1W0j7AXnMARCHe44kbGk65nzi0nXzvl65irnw4hQqFnu907f90IuOd_Frn2BPRc85EKVP4nGlXNtuZ_Qwoc0PoaG1A2fAN-aKKl69bPVZnjZoSNbN4cLTTrND-lWbosK-dl1Y5ziVub_H_gCk-hFY8VFWr90FA4cjeLOZrrJDP1-ts_46f3F_ROufhBpx7upv6lUF_wObCuWZs.LgPmrD5edGDNjrIftBUg0l-00BQpGHBcA7O6ZvEUWhc&dib_tag=se&keywords=macbook%2Bcharger&qid=1770032118&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1

=== Product 2 ===
  Title: N/A...
  Price: 39.99
  Rating: 4.2
  URL: https://www.amazon.com/M1-M4%E3%80%90Original-Quality%E3%80%91Type-Magnetic-Applicable-2021-2025/dp/B0F1FCDW19/ref=sr_1_2_sspa?dib=eyJ2IjoiMSJ9.ruSU4Gshj7Qto5dVEGD7y5KR1OH1-1W0j7AXnMARCHe44kbGk65nzi0nXzvl65irnw4hQqFnu907f90IuOd_Frn2BPRc85EKVP4nGlXNtuZ_Qwoc0PoaG1A2fAN-aKKl69

---
## Test 6: Manual Job Control (Trigger/Status/Fetch)

For long-running operations, you can manually control the workflow.

In [5]:
import asyncio

PRODUCT_URL = "https://www.amazon.com/dp/B0CRMZHDG8"

print(f"Manual workflow for: {PRODUCT_URL}\n")

async with client:
    # Step 1: Trigger the scrape
    print("Step 1: Triggering scrape...")
    job = await client.scrape.amazon.products_trigger(url=PRODUCT_URL)
    print(f"  Job ID: {job.snapshot_id}")
    
    # Step 2: Poll for status
    print("\nStep 2: Polling for status...")
    while True:
        status = await job.status()
        print(f"  Status: {status}")
        if status == "ready":
            break
        elif status in ("error", "failed"):
            print("  Job failed!")
            break
        await asyncio.sleep(5)
    
    # Step 3: Fetch results
    if status == "ready":
        print("\nStep 3: Fetching results...")
        data = await job.fetch()
        if data:
            item = data[0] if isinstance(data, list) else data
            print(f"  Title: {item.get('title', 'N/A')}")
            print(f"  Price: {item.get('final_price', 'N/A')}")
            print(f"  Rating: {item.get('rating', 'N/A')}")

Manual workflow for: https://www.amazon.com/dp/B0CRMZHDG8

Step 1: Triggering scrape...
  Job ID: sd_ml53gjm31eqjyimt4j

Step 2: Polling for status...
  Status: running
  Status: running
  Status: ready

Step 3: Fetching results...
  Title: STANLEY Quencher H2.0 Tumbler with Handle and Straw 40 oz | Flowstate 3-Position Lid | Cup Holder Compatible for Travel | Insulated Stainless Steel Cup | BPA-Free | Fuchsia
  Price: 45
  Rating: 4.7
